In [35]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import numpy as np

import plotly.graph_objs as go
import chart_studio.plotly as py
import cufflinks
from plotly.offline import iplot
cufflinks.go_offline()
# Set global theme
cufflinks.set_config_file(world_readable=True, theme='white')

In [36]:
# Parámetros a modificar

#modelo = "Volkswagen"
marca = "Volkswagen"
modelo = "gol"
año = "2010"
url = "https://www.chileautos.cl/vehiculos/?q=(And.(C.Marca.{}._.Modelo.{}.)_.TipoVehículo.Autos%2c+camionetas+y+4x4._.Servicio.ChileAutos._.Ano.range({}..).)".format(marca, modelo, año)
print(url)


https://www.chileautos.cl/vehiculos/?q=(And.(C.Marca.Volkswagen._.Modelo.gol.)_.TipoVehículo.Autos%2c+camionetas+y+4x4._.Servicio.ChileAutos._.Ano.range(2010..).)


In [28]:
# headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
# req = requests.get(url=url, headers=headers)

# soup = BeautifulSoup(req.content, "html.parser")
# dataSet = soup.find_all("div", {"class": "listing-item showcase"}) #tag que contiene la info de cada auto




In [37]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
req = requests.get(url=url, headers=headers)

soup = BeautifulSoup(req.content, "html.parser")
dataSet = soup.find_all("div", {"class": "listing-item showcase"}) #tag que contiene la info de cada auto

infoSet = []

seguir = True
pagina = 1

while seguir == True: 
    print("Pagina: ", pagina)
    for carro in dataSet:
        d = {}

        try:
            titulo = carro.find("a", {"class": "js-encode-search"}).text.strip()
        except:
            titulo = None

        if (marca in titulo): # pq a veces hay ofertas promoted en medio de los resultados
            try:
                d["Precio"] = carro.find("div", {"class": "price"}).text.strip()
            except:
                d["Precio"] = None

            try:
                d["Titulo"] = carro.find("a", {"class": "js-encode-search"}).text.strip()
            except:
                d["Titulo"] = None

            try:
                d["Kilometraje"] = carro.find("div", {"data-type": "Odometer"}).text.strip()
            except:
                d["Kilometraje"] = None
                
            try:
                d["Combustible"] = carro.find("div", {"data-type": "Fuel Type"}).text.strip()
            except:
                d["Combustible"] = None    
            
            try:
                link_html = carro.find("a", {"class": "btn btn-primary js-encode-search"})
                d["Link"] = "https://www.chileautos.cl" + link_html.get('href')
            except:
                d["Link"] = None
                
            infoSet.append(d)

    try:
        nextPage = soup.find_all("a", {"class": "page-link next"})
        nextPage = nextPage[0].get('href')
        nextPage = nextPage[1:]
        req = requests.get(url="https://www.chileautos.cl/" + nextPage, headers=headers)
        #time.sleep(1)
        soup = BeautifulSoup(req.content, "html.parser")
        dataSet = soup.find_all("div", {"class": "listing-item standard"})
        pagina += 1
        
    except: # esto pasa cuando no encuentra 
        seguir = False
        print("Terminado")
        
    

Pagina:  1
Pagina:  2
Pagina:  3
Pagina:  4
Pagina:  5
Pagina:  6
Pagina:  7
Pagina:  8
Pagina:  9
Pagina:  10
Pagina:  11
Pagina:  12
Pagina:  13
Pagina:  14
Pagina:  15
Terminado


In [40]:
# transformar la lista a dataFrame
df = pd.DataFrame(infoSet)

In [41]:
df["Kilometraje"].fillna(np.nan, inplace=True)
df.dropna(axis=0, inplace=True, subset = ["Kilometraje"])
df["Precio"].fillna(np.nan, inplace=True)
df.dropna(axis=0, inplace=True, subset = ["Precio"])

In [42]:
# limpiar el precio y kilometraje
df['Precio'] = df['Precio'].str.replace('CLP', '').str.replace('$', '').str.replace('.', '').str.strip()
df['Kilometraje'] = df['Kilometraje'].str.replace('km', '').str.replace('KM', '').str.replace('.', '').str.strip()
df['Precio'] = df['Precio'].astype('int')
df['Kilometraje'] = df['Kilometraje'].astype('int')
    

In [43]:
df

,Precio,Titulo,Kilometraje,Combustible,Link
0,4390000,2013 Volkswagen Gol 1.6 Power 2AB,74000,Bencina,https://www.chileautos.cl/vehiculos/detalles/2...
1,5200000,2018 Volkswagen Gol 1.6 Manual Trendline,69373,Bencina,https://www.chileautos.cl/vehiculos/detalles/2...
2,7190000,2018 Volkswagen Gol 1.6 Manual Power,13000,Bencina,https://www.chileautos.cl/vehiculos/detalles/2...
3,6990000,2019 Volkswagen Gol 1.6 Comfortline,8000,Bencina,https://www.chileautos.cl/vehiculos/detalles/2...
4,6650000,2019 Volkswagen Gol 1.6 Highline,25900,Bencina,https://www.chileautos.cl/vehiculos/detalles/2...
...,...,...,...,...,...
151,8090000,2020 Volkswagen GOL Nuevo Gol Highline 1.6,100,Bencina,https://www.chileautos.cl/vehiculos/detalles/2...
152,6190000,2018 Volkswagen GOL GOL HB 1.6,10036,Bencina,https://www.chileautos.cl/vehiculos/detalles/2...
153,5990000,2018 Volkswagen GOL GOL POWER 1.6,68000,Bencina,https://www.chileautos.cl/vehiculos/detalles/2...
154,6990000,2019 Volkswagen GOL Nuevo Gol Highline 1.6,10000,Bencina,https://www.chileautos.cl/vehiculos/detalles/2...


In [44]:
df.describe()

,Precio,Kilometraje
count,1.480000e+02,148.000000
mean,5.224210e+06,60328.493243
std,1.382870e+06,46425.206136
min,3.100000e+03,20.000000
25%,4.190000e+06,19750.000000
50%,5.395000e+06,54739.000000
75%,6.190000e+06,88000.000000
max,8.590000e+06,270000.000000


In [45]:
# Crear columna con el año
df['Año'] = df['Titulo'].str[:4].astype(int)

In [234]:
# para solo bencina
df[df["Combustible"] == "Bencina"].describe()

,Precio,Kilometraje
count,49,49
mean,10814898,44718
std,2108162,22596
min,6900000,3569
25%,9290000,27700
50%,10290000,42840
75%,12790000,63000
max,15000000,92700


In [51]:
df.to_excel("lista_autos.xlsx")

In [46]:
df.iplot(kind='scatter',
         mode='markers',
         size=10,
         symbol='x',
         colorscale='paired',
         x = 'Año',
         y = 'Precio',
         title='Scatter de año y precio')

In [50]:
df.loc[df['Año']==2016, 'Link' ]

15     https://www.chileautos.cl/vehiculos/detalles/2...
64     https://www.chileautos.cl/vehiculos/detalles/2...
67     https://www.chileautos.cl/vehiculos/detalles/2...
70     https://www.chileautos.cl/vehiculos/detalles/2...
72     https://www.chileautos.cl/vehiculos/detalles/2...
80     https://www.chileautos.cl/vehiculos/detalles/2...
94     https://www.chileautos.cl/vehiculos/detalles/2...
111    https://www.chileautos.cl/vehiculos/detalles/2...
124    https://www.chileautos.cl/vehiculos/detalles/2...
137    https://www.chileautos.cl/vehiculos/detalles/2...
138    https://www.chileautos.cl/vehiculos/detalles/2...
Name: Link, dtype: object